## All the basic preprocessing in one place

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import re


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leapfrog\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Leapfrog\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\Leapfrog\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv(r"C:\Users\Leapfrog\Desktop\Others\Mtech AI\AIAC_536_NLP\AIAC_536_NLP\NLP Assignment 2\zomato_reviews.csv")
df.head(3)

,Review,sentiment
0,Virat Kohli did a great thing to open his rest...,positive
1,This place have some really heathy options to ...,positive
2,Aerocity is the most finest place in Delhi for...,positive


In [3]:
corpus = pd.Series(df.Review.tolist()).astype(str)
corpus

0       Virat Kohli did a great thing to open his rest...
1       This place have some really heathy options to ...
2       Aerocity is the most finest place in Delhi for...
3       Yesterday evening there was small team lunch ,...
4       I find aerocity to be the best place in delhi ...
                              ...                        
1591    || DESI LANE || So we were at alipore's most h...
1592    "Desi Lane" is one of the most trending place ...
1593    One of the cool and pocket pinch restaurant at...
1594    "DESI LANE" one of the best places in town and...
1595    Looking for good place for lunch but dont wann...
Length: 1596, dtype: object

## Text Cleaning (Removal of special characters/punctuations & case folding)


In [4]:
def text_clean(corpus, keep_list):
    '''
    Purpose : Function to keep only alphabets, digits and certain words (punctuations, qmarks, tabs etc. removed)
    
    Input : Takes a text corpus, 'corpus' to be cleaned along with a list of words, 'keep_list', which have to be retained
            even after the cleaning process
    
    Output : Returns the cleaned text corpu
    
    '''
    cleaned_corpus = pd.Series()
    for row in corpus:
        qs = []
        
        for word in row.split():
            if word not in keep_list:
                p1= re.sub(pattern='[^a-zA-Z0-9]', repl=' ',string=word)
                p1 = p1.lower()
                qs.append(p1)
            else : qs.append(word)
        cleaned_corpus = cleaned_corpus._append(pd.Series(' '.join(qs)))
    return cleaned_corpus

## Stopwords Removal

In [5]:
def stopwords_removal(corpus):
    wh_words = ['who', 'what', 'when', 'why', 'how', 'which', 'where', 'whom']
    stop = set(stopwords.words('english'))
    for word in wh_words:
        stop.remove(word)
    corpus = [[x for x in x.split() if x not in stop] for x in corpus]
    return corpus

## Lemmatization

In [6]:
def lemmatize(corpus):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    return corpus

## Stemming

In [7]:
def stem(corpus, stem_type =  None):
    if stem_type == 'snowball':
        stemmer = SnowballStemmer(language = 'english')
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    else:
        stemmer = PorterStemmer()
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    return corpus

In [8]:
def preprocess(corpus, keep_list, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):
    '''
    Purpose : Function to perform all pre-processing tasks (cleaning, stemming, lemmatization, stopwords removal etc.)
    
    Input : 
    'corpus' - Text corpus on which pre-processing tasks will be performed
    'keep_list' - List of words to be retained during cleaning process
    'cleaning', 'stemming', 'lemmatization', 'remove_stopwords' - Boolean variables indicating whether a particular task should 
                                                                  be performed or not
    'stem_type' - Choose between Porter stemmer or Snowball(Porter2) stemmer. Default is "None", which corresponds to Porter
                  Stemmer. 'snowball' corresponds to Snowball Stemmer
    
    Note : Either stemming or lemmatization should be used. There's no benefit of using both of them together
    
    Output : Returns the processed text corpus
    
    '''
    
    if cleaning == True:
        corpus = text_clean(corpus, keep_list)
    
    if remove_stopwords == True:
        corpus = stopwords_removal(corpus)
    else :
        corpus = [[x for x in x.split()] for x in corpus]
    
    if lemmatization == True:
        corpus = lemmatize(corpus)
        
        
    if stemming == True:
        corpus = stem(corpus, stem_type)
    
    corpus = [' '.join(x) for x in corpus]        

    return corpus

In [9]:
common_dot_words = ['U.S.A', 'Mr.', 'Mrs.', 'D.C.']


In [10]:
# Preprocessing with Lemmatization here
corpus_with_lemmatization = preprocess(corpus, keep_list = common_dot_words, stemming = False, stem_type = None, lemmatization = True, remove_stopwords = True)

In [11]:
# Preprocessing with Stemming here here
corpus_with_stemming = preprocess(corpus, keep_list = common_dot_words, stemming = True, stem_type = "snowball", lemmatization = False, remove_stopwords = True)


## Let's see the results on applying


In [13]:
print("Original string: ", corpus[0])

Original string:  Virat Kohli did a great thing to open his restaurant in an exquisite place of Delhi. Wide range of food with lots and lots of options on drinks. Courteous staff with a quick response on anything.


In [15]:
print("String after lemmatiization: ", corpus_with_lemmatization[0])


String after lemmatiization:  virat kohli great thing open restaurant exquisite place delhi wide range food lot lot options drink courteous staff quick response anything


In [17]:
print("String after stemming: ", corpus_with_stemming[0])

String after stemming:  virat koh great thing open restaur exquisit place delhi wide rang food lot lot option drink courteous staff quick respons anyth
